Copyright (C) 2020-2024 - Raytheon BBN Technologies Corp.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0.

Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an
"AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND,
either express or implied. See the License for the specific
language governing permissions and limitations under the License.

Distribution Statement "A" (Approved for Public Release,
Distribution Unlimited).

This material is based upon work supported by the Defense
Advanced Research Projects Agency (DARPA) under Contract No.
HR001119C0102.  The opinions, findings, and conclusions stated
herein are those of the authors and do not necessarily reflect
those of DARPA.


# Timeframe Anomaly Clustering

In [ ]:
import ipaddress
import matplotlib.pyplot as plt
import time
import yaml
from src.TrafficShapeAnalysis import TrafficShapeAnalysis
from sklearn.metrics.pairwise import euclidean_distances
import pandas as pd
import numpy as np
import sys

## User Defined Variables

In the parameter file `config.yml`, Specify the user defined parameters. The descriptions of the paramters are in the file

Run the cell below to load the parameters from `config.yml` and to populate a set of destination addresses from a destination subnet.

In [ ]:
# Load parameters from config.yml - Parameter descriptions are specified in the yaml file
with open('config.yml', 'r') as file:
    params = yaml.safe_load(file)


## Initialization of Variables and Function Declarations

In [ ]:
debug = params['debug']
save_figs = params['save_figs']
compute_features = params['compute_features']
previous_features = str(params['previous_features'])
clustering_alg = str(params['clustering_alg'])
num_clusters = params['num_clusters'] 
quantile = params['quantile'] 
linkage = params['linkage'] 
save_features = params['save_features']
limit_val = params['limit_val']
output_dir_path = params['output_dir_path']

# Transform the list of subnets to a set of IP addresses
destination_addresses = set()
for i in params['destination_subnets']:
    ll = [str(ip) for ip in ipaddress.IPv4Network(i)]
    destination_addresses.update(ll)

vals_of_interest = ['val0', 'val1', 'val2', 'val3', 'val4', 'delta0', 'delta1', 'delta2', 'delta3', 'delta4']

if(output_dir_path[len(output_dir_path)-1] != '/'):
    output_dir_path = output_dir_path + '/'

In [ ]:
# Function used for sampling values from a specific 
def sample_cluster_pca_vals(vals_of_interest, sources, pca_values, limit_val):
    
    curr_cluster_vals = pd.DataFrame(columns = vals_of_interest)
    
    # Sample upto limit_val packets for each source
    for source in sources:
        
        curr_source = pca_values[pca_values['ip'] == source]
        if len(curr_source) > limit_val:
            curr_source = curr_source.sample(limit_val)
    
        curr_cluster_vals = pd.concat([curr_cluster_vals, curr_source[vals_of_interest]], axis=0)
        
    return curr_cluster_vals

## Get the Features Using `TrafficShapeAnalysis` Object

Run the cell below to either compute or load previously computed Features

If you are computing the features this will initialize a new `TrafficShapeAnalysis` object and then compute the features This can take a long time because for each packet, features are being calculated and the data is undergoing interpolation and dimensionality reduction. While the object is initializing, you can see how many packets with the specified protocol and destination subnet have been processed.

If you are loading previously computed features, it will load them into a new `TrafficShapeAnalysis` object and it could be very fast or very slow depending on how many points you have in the data.

In [ ]:

init_time = time.time()

# Compute Features
if compute_features == 1:
    print('Computing Features')
    traffic_analysis = TrafficShapeAnalysis(path_to_csv=params['path_to_csv'],
                                            csv_filenames=params['csv_files'],
                                            output_filename=params['output_file'],
                                            destination_sources=destination_addresses,
                                            traffic_type=params['traffic_type'],
                                            max_states=params['max_states'],
                                            interpolation_n=params['interpolation_n'],
                                            show_pc_plot=False)

    if debug == 1:
        print(f"Total time: {time.time() - init_time}")
        
#Load previously computed features
else:
    print('Loading Previous Features')
    traffic_analysis = TrafficShapeAnalysis(pc_values_csv=previous_features)

## Cluster the Data
The data can be clustered using agglomerative, mean shift, or spectral clustering. The selection of the algorithm and setting the algorithm parameters is done in the configuration file `config.yml`

1. <b> Mean Shift Clustering:</b> 
The `quantile` parameter must be specified.

2. <b> Agglomerative Clustering:</b> 
The `n_clusters` and the `linkage` parameters must be specificed. Appropriate values for `linkage` are 'single', 'average', or 'complete' ('average' recommended if you don't want to optimize the parameters).

3. <b> Spectral Clustering: </b> 
The `n_clusters` parameter must be specified.


This will print clustering metrics, including the Silhouette Coefficient and the Davies-Bouldin Index.

In [ ]:
print(f'Running {clustering_alg} clustering')
if clustering_alg == "MeanShift":
    # Mean shift clustering
    cluster_dict = traffic_analysis.cluster(clustering_alg, quantile=quantile, in_dict=True, labels=True, return_cluster_object=True)
elif clustering_alg == "Agglomerative":
    cluster_dict = traffic_analysis.cluster(clustering_alg, n_clusters=num_clusters, linkage=linkage, in_dict=True, labels=True)
elif clustering_alg == "Spectral":
    cluster_dict = traffic_analysis.cluster(clustering_alg, n_clusters=num_clusters, in_dict=True, labels=True)
else:
    sys.exit("Please specify the correct algorithm")
    
if debug == 1:
    print('cluster_dict: ', cluster_dict)

## Plotting the Data

In [ ]:
cluster_numbers = sorted(list(cluster_dict.keys()))
if debug == 1:
    print("Cluster Numbers = ", cluster_numbers)

### Bar Charts of Number of Source Addresses in each Cluster
Run the cell below to view how many source addresses are in each cluster.

In [ ]:
# Plotting bar charts

source_counts = [len(cluster_dict[n]) for n in cluster_numbers]

plt.bar(cluster_numbers, source_counts, alpha=0.5, align='center')
plt.ylabel('Count of Source Addresses')
plt.xlabel('Cluster Number')

if clustering_alg == "MeanShift":
    plt.title(f'Sources per Cluster with {clustering_alg} Clustering, quantile = {quantile}')
    plt.tight_layout()
    if save_figs == 1:
        plt.savefig(f'{output_dir_path}{clustering_alg}_quantile_{quantile}.png')
elif clustering_alg == "Agglomerative":
    plt.title(f'Sources per Cluster with {clustering_alg} Clustering, num_clusters ={num_clusters} and linkage = {linkage}')
    plt.tight_layout()
    if save_figs == 1:
        plt.savefig(f'{output_dir_path}{clustering_alg}_num_clusters_{num_clusters}_linkage_{linkage}.png')
elif clustering_alg == "Spectral":
    plt.title(f'Sources per Cluster with {clustering_alg} Clustering, num_clusters ={num_clusters}')
    plt.tight_layout()
    if save_figs == 1:
        plt.savefig(f'{output_dir_path}{clustering_alg}_num_clusters_{num_clusters}.png')


plt.show()


In [ ]:
pca_values = traffic_analysis.sources_pcs_df
# Axis labels, which are the first three principal component values.
idx, idy, idz = "val0", "val1", "val2"

plot_vals_of_interest = [idx, idy, idz]

num_clusters = len(cluster_numbers)

x_min = pca_values[idx].min()
x_max = pca_values[idx].max()
y_min = pca_values[idy].min()
y_max = pca_values[idy].max()
z_min = pca_values[idz].min()
z_max = pca_values[idz].max()



### Individual Clusters in 3D
Each cluster can be visualized in 3D using the first three principal components (PCs). These PCs do not have any meaning, but they hold the most meaningful parts (most variation) of the original data.

Note: some clusters may look similar, however, they may still be different as all the values are not being plotted here. Internally 5 principal components are beiny used to compute the clusters. However, only the top 3 most important principal components are being plotted

Note: If there are too many packets/source, then only the max `limit_val` packets from config file are plotted for each source

Run the cell that plots the data. You should get several plots, one for each cluster.

In [ ]:
# Plotting each cluster separately

for cluster_n in cluster_numbers:
    sources = cluster_dict[cluster_n]
#     print(sources)
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')

    # Plot up to 200 packets for each source
    for source in sources:
        curr_pca_values = pca_values[pca_values['ip'] == source]
        if len(curr_pca_values) > limit_val:
            curr_pca_values = curr_pca_values.sample(limit_val)

        ax.scatter(curr_pca_values[idx], curr_pca_values[idy], curr_pca_values[idz], s=5, label=source)

    ax.set_xlabel(idx)
    ax.set_ylabel(idy)
    ax.set_zlabel(idz)

    ax.axes.set_xlim3d(left=x_min, right=x_max)
    ax.axes.set_ylim3d(bottom=y_min, top=y_max)
    ax.axes.set_zlim3d(bottom=z_min, top=z_max)

    plt.legend()

    handles, labels = ax.get_legend_handles_labels()
    # Sort both labels and handles by labels
    labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
    ax.legend(handles, labels)
    
    if clustering_alg == "MeanShift":
        plt.title(f"{clustering_alg} Clustering, quantile = {quantile}', Cluster {cluster_n}, Sources: {len(sources)}")
        if save_figs == 1:
            plt.savefig(f'{output_dir_path}{clustering_alg}_quantile_{quantile}cluster_n{cluster_n}.png')
    elif clustering_alg == "Agglomerative":
        plt.title(f"{clustering_alg} Clustering, num_clusters ={num_clusters}, linkage = {linkage}, Cluster {cluster_n}, Sources: {len(sources)}")
        if save_figs == 1:
            plt.savefig(f'{output_dir_path}{clustering_alg}_num_clusters_{num_clusters}_linkage_{linkage}_cluster_n{cluster_n}.png')
    elif clustering_alg == "Spectral":
        plt.title(f"{clustering_alg} Clustering, num_clusters ={num_clusters}, Cluster {cluster_n}, Sources: {len(sources)}")
        if save_figs == 1:
            plt.savefig(f'{output_dir_path}{clustering_alg}_num_clusters_{num_clusters}_cluster_n{cluster_n}.png')
    


### All Clusters in 3D
Clusters can be visualized in 3D using the first three principal components (PCs). These PCs do not have any meaning, but they hold the most meaningful parts (most variation) of the original data.

Note: some clusters may look similar, however, they may still be different as all the values are not being plotted here. Internally 5 principal components are beiny used to compute the clusters. However, only the top 3 most important principal components are being plotted

Note: If there are too many packets/source, then only the max `limit_val` packets from config file are plotted for each source

Run the cell that plots the data. You should get with one plot of all the clusters

In [ ]:
# Plotting all the clusters together

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

for cluster_n in cluster_numbers:
    sources = cluster_dict[cluster_n]
    sample_cluster_vals = sample_cluster_pca_vals(plot_vals_of_interest, sources, pca_values, limit_val)
    ax.scatter(sample_cluster_vals[idx], sample_cluster_vals[idy], sample_cluster_vals[idz], s=5, label=f'Cluster {cluster_n}')

ax.set_xlabel(idx)
ax.set_ylabel(idy)
ax.set_zlabel(idz)

ax.axes.set_xlim3d(left=x_min, right=x_max)
ax.axes.set_ylim3d(bottom=y_min, top=y_max)
ax.axes.set_zlim3d(bottom=z_min, top=z_max)

plt.legend()

handles, labels = ax.get_legend_handles_labels()
# Sort both labels and handles by labels
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles, labels)
plt.title(f"All the {num_clusters} clusters with {clustering_alg} Clustering")

if clustering_alg == "MeanShift":
    plt.title(f"All the {num_clusters} {clustering_alg} clusters")
    if save_figs == 1:
        plt.savefig(f'{output_dir_path}{clustering_alg}_quantile_{quantile}_all_Clusters.png')
elif clustering_alg == "Agglomerative":
    plt.title(f"All the {num_clusters} {clustering_alg} clusters")
    if save_figs == 1:
        plt.savefig(f'{output_dir_path}{clustering_alg}_num_clusters_{num_clusters}_linkage_{linkage}_all_Clusters.png')
elif clustering_alg == "Spectral":
    plt.title(f"All the {num_clusters} {clustering_alg} clusters")
    if save_figs == 1:
        plt.savefig(f'{output_dir_path}{clustering_alg}_num_clusters_{num_clusters}_all_Clusters.png')

## Exporting the Features (Principal Component Values)
Processing the data and calculating the features can be a time consuming process, especially for huge sets of data. To save the data for later use, you can export it to a CSV file by running the cell that does this. The CSV file will be in the `path_to_csv` directory with the `output_filename` specified previously.

The CSV file will contain information for each packet and its principal component values. The columns are specified in the first line of the CSV file.

In [ ]:
# Exporting the principal component values
if save_features == 1:
    print('Saving the features File')
    traffic_analysis.get_sources_pcs_csv()
    

## Cluster Analysis

Different ways of analyzing how the clusters are spread

### Cluster Center Distances
Here all the packets in a cluster are used to compute the cluster center (Average). Then the distance from each cluster center to the other Cluster center is computed

In [ ]:
print("Computing Cluster Center Distances")



if num_clusters > 1: 
    dist_df = pd.DataFrame(np.nan, index=cluster_numbers, columns=cluster_numbers)
    
    for cluster_n in cluster_numbers:
    #     print(f"cluster_n {cluster_n}")
        current_cluster = pca_values[pca_values['ip'].isin(cluster_dict[cluster_n])]
        current_cluster_vals = current_cluster[vals_of_interest]
    #     curr_cluster_mean = current_cluster_vals.mean().values.tolist()
        curr_cluster_mean = current_cluster_vals.mean().values
    #     print('curr_cluster_mean: ', curr_cluster_mean)
        dist_df.loc[cluster_n, cluster_n] = 0

        if cluster_n < num_clusters-1:

            for other_cluster_n in range(cluster_n+1, num_clusters):

                other_cluster = pca_values[pca_values['ip'].isin(cluster_dict[other_cluster_n])]
                other_cluster_vals = other_cluster[vals_of_interest]
                other_cluster_mean = other_cluster_vals.mean().values
                cluster_center_dist = euclidean_distances([curr_cluster_mean], [other_cluster_mean]).ravel()
                dist_df.loc[cluster_n, other_cluster_n] = cluster_center_dist
                dist_df.loc[other_cluster_n, cluster_n] = cluster_center_dist


    print("\nCluster Center Distances: Distance from each cluster center (average) to the other cluster centers")
    display(dist_df)
else:
    print('Only one cluster. No distances to Compute')
    


In [ ]:
print("Computing Average Cluster Center Distances")

if num_clusters > 1: 
    for d in range(0, len(dist_df)):
        row = dist_df.loc[d, :].to_numpy().nonzero()
        non_zero_dists = dist_df.loc[d, row].mean()
        print('Cluster: ', d, ' Average Distance to Other Clusters: ', non_zero_dists)
else:
    print('Only one cluster. No distances to Compute')
    

### Cluster Points Distances
Here up to `limit_val` (in config file) packets/source in a cluster are used to compute all the distances from these packets to the packets in other clusters. The distances are then averaged to compute the average distance from a cluster to the other clusters

In [ ]:
print("Computing Cluster Points Distances")

if num_clusters > 1: 
    cluster_point_dist_df = pd.DataFrame(np.nan, index=cluster_numbers, columns=cluster_numbers)


    for cluster_n in cluster_numbers:
        sources = cluster_dict[cluster_n]
    #     if debug == 1:
    #         print('Processing cluster: ',  cluster_n)

        # Find distances between points of current cluster to points in other dists
        current_cluster_vals = sample_cluster_pca_vals(vals_of_interest, sources, pca_values, limit_val)
        curr_dists = list(euclidean_distances(current_cluster_vals, current_cluster_vals).ravel())
        curr_dists =  [d for d in curr_dists if d > 0] # remove the distances from the point to itself
        cluster_point_dist_df.loc[cluster_n, cluster_n] = round(np.average(curr_dists), 2)

        if cluster_n < num_clusters-1:

            for other_cluster_n in range(cluster_n+1, num_clusters):

                sources = cluster_dict[other_cluster_n]
                other_cluster_vals = sample_cluster_pca_vals(vals_of_interest, sources, pca_values, limit_val)
                curr_dists = list(euclidean_distances(current_cluster_vals, other_cluster_vals).ravel())
                cluster_point_dist_df.loc[cluster_n, other_cluster_n] = round(np.average(curr_dists), 2)
                cluster_point_dist_df.loc[other_cluster_n, cluster_n] = round(np.average(curr_dists), 2)

    print("\nCluster Points Distances: averaged distances from packets in one cluster to the packets in other clusters")
    display(cluster_point_dist_df)

else:
    print('Only one cluster. No distances to Compute')
    